In [1]:
import d6tflow
import pandas as pd

from d6tflow.functional import Flow
flow = Flow()

Welcome to d6tflow!


## Define workflow

In [2]:
@flow.step(d6tflow.tasks.TaskCache)
def get_data1(task):
    df = pd.DataFrame({'a':range(3)})
    task.save(df)

In [3]:
@flow.step(d6tflow.tasks.TaskCache)
def get_data2(task):
    df = pd.DataFrame({'a':range(3)})
    task.save(df)

In [4]:
@flow.step(d6tflow.tasks.TaskPqPandas)
@flow.requires({"input1":get_data1, "input2":get_data2})
def use_data(task):
    data = task.inputLoad()
    df1 = data['input1']
    df2 = data['input2']
    df3 = df1.join(df2, lsuffix='1', rsuffix='2')
    df3['b']=df3['a1']*task.multiplier # use task parameter
    task.save(df3)

## Execute workflow

In [5]:
flow.add_params({'multiplier': d6tflow.IntParameter(default=0)})
use_params = {'multiplier':4}

In [6]:
flow.preview(use_data, params=use_params)


 ===== Luigi Execution Preview ===== 


└─--[use_data-{'multiplier': '4'} (PENDING)]
   |--[get_data1- (PENDING)]
   └─--[get_data2- (PENDING)]

 ===== Luigi Execution Preview ===== 



In [7]:
flow.run(use_data, params=use_params, forced_all_upstream=True, confirm=False)

no tasks to invalidate

===== Luigi Execution Summary =====

Scheduled 3 tasks of which:
* 3 ran successfully:
    - 1 get_data1(multiplier=4)
    - 1 get_data2(multiplier=4)
    - 1 use_data(multiplier=4)

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



In [8]:
flow.outputLoad(get_data1, params=use_params)

,a
0,0
1,1
2,2


In [9]:
flow.outputLoad(use_data, params=use_params)

,a1,a2,b
0,0,0,0
1,1,1,4
2,2,2,8
